In [7]:
!pip install opencv-python





FileNotFoundError: [WinError 2] The system cannot find the file specified

In [1]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as compare_ssim
import tkinter as tk
from tkinter import filedialog, messagebox,simpledialog
import os

currency_paths = {
    "2000": {
        "front": "S:\\currency_image\\originalfront2000.jpeg",
        "back": "S:\\currency_image\\originalback2000.jpeg"
    },
    "500": {
        "front": "S:\\currency_image\\originalfront500.jpg",
        "back": "S:\\currency_image\\originalback500.jpg"
    },
    "100": {
        "front": "S:\\currency_image\\originalfront100.jpg",
        "back": "S:\\currency_image\\originalback100.jpg"
    },
    "200": {
        "front": "S:\\currency_image\\originalfront200.jpeg",
        "back": "S:\\currency_image\\originalback200.jpeg"
    },
}

# Reading and processing the set of images
def process(img, img1):
    res1 = cv2.resize(img, (800, 300))
    res2 = cv2.resize(img1, (800, 300))
    image = np.concatenate((res1, res2), axis=0)
    img_median = cv2.medianBlur(image, 3)
    gray = cv2.cvtColor(img_median, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 60, 180)
    return edges

def select_front_image():
    # Open a file dialog to select the front image
    front_path = filedialog.askopenfilename()
    if front_path:
        front_entry.delete(1.0, tk.END)
        front_entry.insert(tk.END, front_path)

def select_back_image():
    # Open a file dialog to select the back image
    back_path = filedialog.askopenfilename()
    if back_path:
        back_entry.delete(1.0, tk.END)
        back_entry.insert(tk.END, back_path)

def display_difference():
    front_image_path = front_entry.get("1.0", tk.END).strip()
    back_image_path = back_entry.get("1.0", tk.END).strip()

    if not front_image_path or not back_image_path:
        messagebox.showerror("Error", "Please select both front and back images.")
        return

    img1 = cv2.imread(front_image_path)
    img2 = cv2.imread(back_image_path)

    if img1 is None or img2 is None:
        messagebox.showerror("Error", "Invalid image file. Please check the file path and format.")
        return

    edges1 = process(img1, img2)

    found_match = False
    matched_denomination = ""
    similarity_threshold = 0.9

    input_denomination = simpledialog.askstring("Denomination", "Enter the denomination (e.g., 100, 500):")
    if input_denomination is None:
        return  # User canceled the input dialog

    paths = currency_paths.get(input_denomination)
    if paths is None:
        messagebox.showerror("Error", "Invalid denomination. Please enter a valid denomination.")
        return

    reference_img1 = cv2.imread(paths['front'])
    reference_img2 = cv2.imread(paths['back'])

    if reference_img1 is not None and reference_img2 is not None:
        reference_edges1 = process(reference_img1, reference_img2)

        (score, diff) = compare_ssim(edges1, reference_edges1, full=True)

        diff = (diff * 255).astype(np.uint8)  # Convert the difference image to uint8

        cv2.imshow('Difference Image', diff)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        if score >= similarity_threshold:
            found_match = True
            matched_denomination = input_denomination

    if found_match:
        messagebox.showinfo("Result", f"The displayed currency is genuine ({matched_denomination})")
    else:
        messagebox.showinfo("Result", "The displayed currency is fake")

root = tk.Tk()
root.title("Currency Checker")

# Front Image Entry
front_label = tk.Label(root, text="Front Image:",fg="black",bg="white",height=3,width=40)
front_label.pack()

front_entry = tk.Text(root, height=2, width=50)
front_entry.pack()

front_button = tk.Button(root, text="Select Front Image", command=select_front_image,fg="white",bg="black",height=3,width=40)
front_button.pack()

# Back Image Entry
back_label = tk.Label(root, text="Back Image:",fg="black",bg="white",height=3,width=40)
back_label.pack()

back_entry = tk.Text(root, height=2, width=50)
back_entry.pack()

back_button = tk.Button(root, text="Select Back Image", command=select_back_image,fg="white",bg="black",height=3,width=40)
back_button.pack()

# Display Difference Button
difference_button = tk.Button(root, text="Display Difference", command=display_difference,fg="white",bg="black",height=3,width=40)
difference_button.pack()

root.mainloop()


ModuleNotFoundError: No module named 'cv2'